# Humpback Whale Identification - CNN with Keras
This kernel is based on [Anezka Kolaceke](https://www.kaggle.com/anezka)'s awesome work: [CNN with Keras for Humpback Whale ID](https://www.kaggle.com/anezka/cnn-with-keras-for-humpback-whale-id)

In [ ]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.image as mplimg
from matplotlib.pyplot import imshow
import gc

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from keras import layers
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from keras.models import Model

import keras.backend as K
from keras.models import Sequential

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

import cv2

In [ ]:
os.listdir("../input/")

In [ ]:
train_df = pd.read_csv("../input/train.csv")
train_df.head(20)

In [ ]:
SIZE = 100

In [ ]:
def prepareImages(data, m, dataset):
    print("Preparing images")
    X_train = np.zeros((m, SIZE, SIZE,3))
    count = 0
    threshold = 100
    for fig in data['Image']:

        img = image.load_img("../input/"+dataset+"/"+fig, target_size=(SIZE, SIZE,3))
        
        #img_data = np.asarray(img)
        #ret, img_thresh = cv2.threshold(img_data, threshold, 255, cv2.THRESH_BINARY)
        #kernel = np.ones((2,2),np.uint8)
        
        #x = cv2.morphologyEx(img_thresh, cv2.MORPH_BLACKHAT, kernel)
        
        x = image.img_to_array(img)
        x = preprocess_input(x)
        
        X_train[count] = x
        if (count%1000 == 0):
            print("Processing image: ", count+1, ", ", fig)
        count += 1
    
    return X_train

In [ ]:
def prepare_labels(y):
    for i in range(len(y)):
        if y[i] == "new_whale":
            y[i] = '1'
        else:
            y[i] = '0'
    values = np.array(y)
    
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    # print(integer_encoded)

    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    # print(onehot_encoded)

    y = onehot_encoded
    # print(y.shape)
    return y, label_encoder

In [ ]:
X = prepareImages(train_df, train_df.shape[0], "train")
X /= 255

In [ ]:
#import tensorflow as tf
#from keras import backend

#rgb = tf.image.grayscale_to_rgb(X[1])
#array = backend.get_session().run(rgb)
#plt.imshow(array)
#plt.show()

In [ ]:
y, label_encoder = prepare_labels(train_df['Id'])
train_df.head(20)

In [ ]:
y.shape

In [ ]:
model = Sequential()

model.add(Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0', input_shape = (SIZE, SIZE, 3)))

model.add(BatchNormalization(axis = 3, name = 'bn0'))
model.add(Activation('relu'))

model.add(MaxPooling2D((2, 2), name='max_pool'))
model.add(Conv2D(64, (3, 3), strides = (1,1), name="conv1"))
model.add(Activation('relu'))
model.add(AveragePooling2D((3, 3), name='avg_pool'))

model.add(Flatten())
model.add(Dense(500, activation="relu", name='rl'))
model.add(Dropout(0.8))
model.add(Dense(2, activation='softmax', name='sm'))

model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(X, y, epochs=15, batch_size=100, verbose=1)
gc.collect()

In [ ]:
plt.plot(history.history['acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()

In [ ]:
test = os.listdir("../input/test/")
print(len(test))

In [ ]:
col = ['Image']
test_df = pd.DataFrame(test, columns=col)
test_df['Id'] = ''

In [ ]:
X = prepareImages(test_df, test_df.shape[0], "test")
X /= 255

In [ ]:
#import tensorflow as tf
#from keras import backend

#rgb = tf.image.grayscale_to_rgb(X[2])
#array = backend.get_session().run(rgb)
#plt.imshow(array)
#plt.show()

In [ ]:
predictions = model.predict(np.array(X), verbose=1)

In [ ]:
print(predictions)
for i, pred in enumerate(predictions):
    if pred[0] > pred[1]:
        ans = 'w_f48451c'
    else:
        ans = 'new_whale'
    test_df.loc[i, 'Id'] = ans

In [ ]:
test_df.head(10)

In [ ]:
test_df.to_csv('submission.csv', index=False)